# Getting Real - Investigating a published ephys dataset
Based on tutorial material by Julia Sprenger

## The Reach-2-Grasp experiment

![R2G_overview](reach_to_grasp_material/R2G_task_overview.png)

Full data manuscript and dataset
- Brochier, T., Zehl, L., Hao, Y., Duret, M., Sprenger, J., Denker, M., Grün, S. & Riehle, A. (2018). Massively parallel recordings in macaque motor cortex during an instructed delayed reach-to-grasp task, Scientific Data, 5, 180055. http://doi.org/10.1038/sdata.2018.55
- https://gin.g-node.org/INT/multielectrode_grasp

### Neuronal data sources
<img src="reach_to_grasp_material/R2G_arrays.jpg" width="500"/>
Black numbers indicate the connector-aligned IDs.

### Data and metadata sources
<img src="reach_to_grasp_material/R2G_files.png" width="700"/>

A compiled version of a part of this dataset including metadata is available in the shared google drive.

### Dataset overview - General
We can directly get an overview of the contained objects from the `neo.Block` print

In [1]:
import neo
filename = 'reach_to_grasp_material/i140703-001.nix'
with neo.io.NixIO(filename, 'ro') as io:
    block = io.read_block()
block

Block with 1 segments
name: 'Reach2Grasp data'
annotations: {'nix_name': 'neo.block.6aa7b5491ab74067aee4183310955892'}
rec_datetime: datetime.datetime(2021, 8, 25, 21, 3, 51)
# segments (N=1)
0: Segment with 1 analogsignals, 1 events, 271 spiketrains
   name: 'Segment 0'
   description: 'Segment containing data from t_start to t_stop'
   annotations: {'nix_name': 'neo.segment.1c1ad7987ca048efa7b433b4e87e0f71',
     'condition': 1}
   # analogsignals (N=1)
   0: AnalogSignal with 96 channels of length 74455; units uV; datatype float32 
      name: 'Channel bundle (chan1,chan2,chan3,chan4,chan5,chan6,chan7,chan8,chan9,chan10,chan11,chan12,chan13,chan14,chan15,chan16,chan17,chan18,chan19,chan20,chan21,chan22,chan23,chan24,chan25,chan26,chan27,chan28,chan29,chan30,chan31,chan32,chan33,chan34,chan35,chan36,chan37,chan38,chan39,chan40,chan41,chan42,chan43,chan44,chan45,chan46,chan47,chan48,chan49,chan50,chan51,chan52,chan53,chan54,chan55,chan56,chan57,chan58,chan59,chan60,chan61,chan62,chan6

This summary already tells us that we only need to take care of a single segment with one event, one analogsignal and multiple spiketrain objects. The 96 continuously sampled channels are sampled a 'low' sampling rate of 1kHz and contain neural data, so data comparable to local-field potential measurements.

### Dataset overview - SpikeTrains
To learn more about the spiketrains, we print the spiketrain annotations:

In [2]:
segment = block.segments[0]
segment.spiketrains[0].annotations

{'nix_name': 'neo.spiketrain.22a41c20c0614a238d32dea2d46673ca',
 'id': 'Unit 1000',
 'channel_id': 1,
 'unit_id': 0,
 'unit_tag': 'unclassified',
 'electrode_reject_HFC': False,
 'electrode_reject_LFC': False,
 'electrode_reject_IFC': False,
 'connector_aligned_id': 93,
 'coordinate_x': array(0.8) * mm,
 'coordinate_y': array(3.6) * mm,
 'sua': False,
 'mua': False,
 'noise': True}


## Dataset overview - AnalogSignals
To learn more about the channels of the analogsignal, we print the entries of the array_annotations of the single analogsignal object:

In [3]:
segment.analogsignals[0].array_annotations.keys()

dict_keys(['channel_names', 'channel_ids', 'file_origin', 'connector_ID', 'connector_pinID', 'nev_dig_factor', 'nb_sorted_units', 'nev_hi_freq_order', 'nev_hi_freq_type', 'nev_lo_freq_order', 'nev_lo_freq_type', 'nsx_hi_freq_order', 'nsx_lo_freq_order', 'nsx_hi_freq_type', 'nsx_lo_freq_type', 'description', 'nsx', 'hi_pass_freq', 'lo_pass_freq', 'hi_pass_order', 'lo_pass_order', 'filter_type', 'electrode_reject_HFC', 'electrode_reject_LFC', 'electrode_reject_IFC', 'connector_aligned_ids', 'coordinates_x', 'coordinates_y'])

We see that both, spiketrains as well as channels are annotated with a 'connector_aligned_id', indicating the spatial source of the signal. In addition the coordinates in x and y direction are provided in physical units for each channel and spiketrain. Spiketrains also carry information about 'noise', 'mua' or 'sua' assignment, indicating that the spikes were spikesorted and assigned to one of the three unit categories:
- *noise*: non-neural threshold crossing events)
- *mua*: multi-unit-activity - neural threshold crossing events that can not be uniquely assigned to a virtual neuron unit
- *sua*: single-unit-activity - neural threshold crossing events that are assigned to a single virtual neuron unit

## Dataset overview - Events
To learn more about the events, we print the labels and annotations of the single event object:

In [4]:
import numpy as np
event = segment.events[0]
print(f'Event object name: {event.name}')
print(f'Number of event times: {len(event)}')
print(f'Unique event labels: {np.unique(event.labels)}')
print(f'Event annotation keys: {event.array_annotations.keys()}')
print(f'Unique trial_ids: {np.unique(event.array_annotations["trial_id"])}')
print(f'Number of reward-on events: {sum(event.labels=="RW-ON")}')

Event object name: TrialEvents
Number of event times: 192
Unique event labels: ['CUE-OFF' 'CUE-ON' 'DO' 'FSRplat-OFF' 'FSRplat-ON' 'GO-ON' 'HEplat-OFF'
 'HEplat-ON' 'OBB' 'OR' 'OT' 'RW-OFF' 'RW-ON' 'RW-ON-REP' 'SR' 'SR-REP'
 'STOP' 'TS-ON' 'WS-ON']
Event annotation keys: dict_keys(['trial_id', 'trial_timestamp_id', 'performance_in_trial', 'performance_in_trial_str', 'belongs_to_trialtype', 'trial_event_labels', 'trial_reject_HFC', 'trial_reject_LFC', 'trial_reject_IFC'])
Unique trial_ids: [ 1  2  3  4  5  6  7  8  9 10 11 12 13]
Number of reward-on events: 11


# Working with a dataset - Neo methods and utility functions

In the following lines we are going to introduce a set of useful features of neo objects while cutting the dataset into individual trial segments. Based on the trial events and their annotations we can identify successful (correct) trials and select the time around the start of the trial.

### Filtering for event objects

In [5]:
events = block.filter(objects='Event', name='TrialEvents')[0]

### Masking Neo objects

First we are selecting event times that are occurring during successful trials, marked by the `performance_in_trial_str` as `correct_trial`. In addition we also only want to select trial start (`TS-ON`) events. Based on the array annotations we are creating two masks and apply both to the original event to generate a new event object via slicing.

In [6]:
mask1 = event.array_annotations['performance_in_trial_str'] == 'correct_trial'
mask2 = event.array_annotations['trial_event_labels'] == 'TS-ON'
correct_TS_event = event[mask1 & mask2]
print(len(correct_TS_event))

11


In [7]:
start_event = correct_TS_event

### Utility function: Epoch creation
In the next step we are making use of neo utility functions to generate epochs around the time points extracted.

Construct analysis epochs from 10ms before the TS-ON of a successful behavioral trial to 15ms after TS-ON. The name "analysis_epochs" is given to the resulting Neo Epoch object. The object is not attached to the Neo Segment. The parameter event2 of add_epoch() is left empty, since we are cutting around a single event, as opposed to cutting between two events.

In [8]:
from neo.utils import add_epoch
import quantities as pq

In [9]:
pre = -10 * pq.ms
post = 15 * pq.ms
epoch = add_epoch(
    segment,
    event1=start_event, event2=None,
    pre=pre, post=post,
    attach_result=False,
    name='analysis_epochs')

The number of epochs generated is the same as the number of trial start events and all epochs have the same duration as we extract 25ms of data around each trial start event.

In [10]:
print('Number of epochs: {}'.format(len(epoch)))
print('Durations of epochs: {}'.format(epoch.durations))

Number of epochs: 11
Durations of epochs: [0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025] s


### Utility function: cutting segment by epochs


Now we use the previously defined epochs to cut the segment containing the complete dataset into subsets, where each new segment corresponds to the starting epoch of a successful trial. For capturing the newly generated segments, we create a new Neo block.

Create new segments of data cut according to the analysis epochs of the 'analysis_epochs' Neo Epoch object. The time axes of all segments are aligned such that each segment starts at time 0 (parameter reset_times); annotations describing the analysis epoch are carried over to the segments. A new Neo Block named "data_cut_to_analysis_epochs" is created to capture all cut analysis epochs.


In [11]:
from neo.utils import cut_segment_by_epoch

cut_trial_block = neo.Block(name="data_cut_to_analysis_epochs")
cut_trial_block.segments = cut_segment_by_epoch(segment, epoch, reset_time=True)

In [12]:
len(cut_trial_block.segments)

11

We can confirm that the resetting of the times during the segmentation process was successfull by checking the new times of the trial start events in the new segments:

In [13]:
print('In the new trial TS is happening at {}'.format([s.events[-1][0] for s in cut_trial_block.segments]))

In the new trial TS is happening at [array(0.01) * s, array(0.01) * s, array(0.01) * s, array(0.01) * s, array(0.01) * s, array(0.01) * s, array(0.01) * s, array(0.01) * s, array(0.01) * s, array(0.01) * s, array(0.01) * s]


### More interesting features
- *`DataObject`*`.time_slice()` - create a new object containing data in a subset of the original time span -- works also for containers!
- *`Object`*`.annotate()` - adding a custom annotation
- *`DataObject`*`.array_annotate()` - adding a custom array annotation (channel / spike based annotation)
- *`DataObject`*`.merge()` - merge data values of two DataObjects
- *`DataObject`*`.concatenate()` - concatenate data values of two DataObjects
- `AnalogSignal.downsample()` - create an AnalogSignal with a lower sampling rate
- *`Object`*`.parents`, e.g. `AnalogSignal.segment` - to navigate in neo object structure
- *`Quantity`*`.rescale()` - rescale the data to a new physical unit, e.g., `.rescale(pq.ms)` to rescale to milliseconds
- *`Quantity`*`.simplified()` - simplifies units to SI units
- *`Quantity`*`.dimensionality.latex` - get latex representation of physical unit (for plotting)
